In [ ]:
import sys
import os
import numpy as np
import igraph as ig
import json
from dotenv import load_dotenv
import os

load_dotenv()

HF_TOKEN = os.getenv("HF_TOKEN")
API_URL = os.getenv("API_URL") 
MODEL = os.getenv("MODEL")

sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

#from simulation import run_simulation, generate_network, _initialize_posts_and_weights

In [ ]:
import dotenv
import os

ENV = dotenv.dotenv_values("./.env")
HF_TOKEN = ENV["HF_TOKEN"]
API_URL = ENV["API_URL"]
MODEL = ENV["MODEL"]


print(API_URL)
print(MODEL)

# load params from PARAMS.json
with open("PARAMS.json", "r") as f:
    PARAMS = json.load(f)

# add HF_TOKEN to PARAMS
PARAMS["HF_TOKEN"] = HF_TOKEN
PARAMS["API_URL"] = API_URL
PARAMS["MODEL"] = MODEL


In [ ]:
PARAMS = {
    "time_decay_rate": np.log(2)/2,  # halving the weight every 2 time steps. change /2 -> /k for halving every k time steps.
    "W_agent_success": 1,
    "W_personal_weights": 1,
    "W_post_success" : 1,
    "noise_level": 0.01
}
timesteps = 100
num_agents = 100
fill_history = 3


G = generate_network(N = num_agents, neighbors = 4, p = 0.2, timesteps = timesteps, fill_history = fill_history)
POSTS, WEIGHTS, READ_MATRIX, LIKES = _initialize_posts_and_weights(G, debug = True)


G, POSTS, WEIGHTS, READ_MATRIX, LIKES = run_simulation(G, WEIGHTS, READ_MATRIX, LIKES, PARAMS, POSTS, fill_history, timesteps)
# this is the first timestep that we are simulating, so this is our "time 0", but we have filled some of the history to have meaningfull pasts.
# print the average number of likes for each agent:
print(np.mean(LIKES, axis=1))